In [1]:
import pandas as pd
import numpy as np

# 1. 构造数据集
# 从名为 'data.txt' 的文件中读取数据，文件以制表符（tab, \t）分隔，创建一个 pandas DataFrame。
data = pd.read_csv('data.txt', sep='\t')

# 2. 清理数据中的额外空格
# 去掉列名中的任何额外空格（防止读取数据时列名两端可能有空格）
data.columns = data.columns.str.strip()


# 去掉数据中所有元素的前后空格，防止数据项中的空格影响计算
data = data.applymap(str.strip) # applymap() 方法会将 str.strip 这个函数应用到 data 的每个单元格上。


# 3. 函数：计算信息熵
# 信息熵用于度量数据集的混乱程度或不确定性。目标列是分类结果（例如‘要不要赖床’），
# 信息熵公式为：H(X) = - Σ (p(x) * log2(p(x)))，其中 p(x) 是类别 x 在数据集中出现的概率。
def entropy(target_col):
    # np.unique 返回目标列中的唯一元素和它们的计数
    elements, counts = np.unique(target_col, return_counts=True)
    # print(elements,counts)
    # 按照信息熵公式计算：对每个类别的出现概率计算熵，并求和
    entropy = -np.sum([(count / len(target_col)) * np.log2(count / len(target_col)) for count in counts])
    return entropy

# 4. 函数：计算条件熵
# 条件熵用于度量在给定某一特征的情况下，目标变量的剩余不确定性。
# 它考虑某个属性的不同取值如何影响目标列的熵。
#↓左边是结果，右边是条件
# H(Y|X) = Σ P(x) * H(Y|X=x)，这里 P(x) 是特征取值的概率，H(Y|X=x) 是给定特征取值后的熵。
def cond_entropy(data, attribute, target):
    # 取得指定属性的唯一值及其频率
    values, counts = np.unique(data[attribute], return_counts=True)
    # 计算条件熵，每个属性值下的熵按照该值的概率加权求和
    weighted_entropy = np.sum([
        (counts[i] / np.sum(counts)) * entropy(
            data.where(data[attribute] == values[i]).dropna()[target]
        ) for i in range(len(values))
    ])
    return weighted_entropy

# 5. 计算目标列的整体信息熵
# 计算'要不要赖床'这一目标变量的整体信息熵（即没有任何条件时目标列的熵）
total_entropy = entropy(data['要不要赖床'])

# 6. 计算每个属性的信息熵、条件熵和信息增益
# 信息增益表示在某个属性下，信息熵的减少程度，公式为：
# 信息增益 = 总信息熵（结果Y的信息熵） - 条件熵
# 属性列表，分别是季节、时间是否过8点、风力情况
attributes = ['季节', '时间已过 8 点', '风力情况']
info_gain = {}

# 7. 遍历每个属性，分别计算其信息熵、条件熵及信息增益
for attr in attributes:
    # 计算每个属性本身的信息熵（即对该特征的熵进行计算）
    attr_entropy = entropy(data[attr])
    # 计算该属性下的条件熵（即给定该属性后的目标熵）
    cond_ent = cond_entropy(data, attr, '要不要赖床')
    # 通过总信息熵减去条件熵，计算该属性对目标的“信息增益”
    gain = total_entropy - cond_ent
    # 将计算结果存储在字典中，便于后续输出
    info_gain[attr] = {'属性信息熵': attr_entropy, '条件熵': cond_ent, '信息增益': gain}

# 8. 输出结果
# 打印整体目标列的总信息熵
print(f'整体目标信息熵: {total_entropy}')
# 打印每个属性的信息熵、条件熵和信息增益
print('每个条件的属性信息熵、条件熵与信息增益:')
for attr, values in info_gain.items():
    print(f'{attr} - 属性信息熵: {values["属性信息熵"]}, 条件熵: {values["条件熵"]}, 信息增益: {values["信息增益"]}')

整体目标信息熵: 0.9182958340544896
每个条件的属性信息熵、条件熵与信息增益:
季节 - 属性信息熵: 1.8879185026711327, 条件熵: 0.5629072918469556, 信息增益: 0.3553885422075339
时间已过 8 点 - 属性信息熵: 0.9798687566511528, 条件熵: 0.7497052018624697, 信息增益: 0.16859063219201986
风力情况 - 属性信息熵: 1.5545851693377997, 条件熵: 0.8008033728697342, 信息增益: 0.11749246118475531


C:\Users\user\AppData\Local\Temp\ipykernel_3036\3534011943.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str.strip) # applymap() 方法会将 str.strip 这个函数应用到 data 的每个单元格上。
